# Imports and installs

In [3]:
!pip install pypots pygrinder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.8/539.8 kB 11.9 MB/s eta 0:00:0000:01


In [4]:
import numpy as np
import polars as pl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pygrinder import mcar, mnar_t, calc_missing_rate
from typing import Dict, Tuple
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import mean_squared_error
import math
from math import sqrt
import random

# Set global seed
SEED = 23
np.random.seed(SEED)
random.seed(SEED)

# Amputation - PyGrinder

In [5]:
class PyGrinderExec:
    def __init__(self, X: np.ndarray):
        """
        Initialize with original 2D data array
        
        Parameters:
        X: Original data with shape (timesteps, features)
        """
        self.original_data = X.copy()
        self.ground_truth_mcar = None
        self.ground_truth_mnar = None
        self.missing_mask_mcar = None
        self.missing_mask_mnar = None
        
    def generate_missing(self, 
                        mcar_prob: float = 0.1,
                        mnar_cycle: float = 20,
                        mnar_pos: float = 10,
                        mnar_scale: float = 0.1) -> Dict[str, np.ndarray]:
        """
        Generate missing values using both MCAR and MNAR-t methods
        
        Parameters:
        mcar_prob: Probability of missing values for MCAR
        mnar_cycle: Cycle parameter for MNAR-t
        mnar_pos: Position parameter for MNAR-t
        mnar_scale: Scale parameter for MNAR-t
        
        Returns:
        Dict containing corrupted datasets and ground truth
        """
        # Reshape to 3D for PyGrinder
        X_3d = self.original_data[np.newaxis, :, :]
        
        # Generate MCAR missing values
        X_mcar = mcar(X_3d, p=mcar_prob)[0]  # Get back to 2D
        self.missing_mask_mcar = np.isnan(X_mcar)
        self.ground_truth_mcar = self.original_data[self.missing_mask_mcar]
        
        # Generate MNAR-t missing values
        X_mnar = mnar_t(X_3d, cycle=mnar_cycle, pos=mnar_pos, scale=mnar_scale)[0]  # Get back to 2D
        self.missing_mask_mnar = np.isnan(X_mnar)
        self.ground_truth_mnar = self.original_data[self.missing_mask_mnar]
        
        return {
            'X_mcar': X_mcar,
            'X_mnar': X_mnar,
            'ground_truth_mcar': self.ground_truth_mcar,
            'ground_truth_mnar': self.ground_truth_mnar,
            'missing_mask_mcar': self.missing_mask_mcar,
            'missing_mask_mnar': self.missing_mask_mnar
        }
    
    def evaluate_imputation(self, 
                          X_imputed_mcar: np.ndarray, 
                          X_imputed_mnar: np.ndarray) -> Dict[str, Dict[str, float]]:
        """
        Evaluate imputation results against ground truth
        
        Parameters:
        X_imputed_mcar: Imputed data for MCAR case
        X_imputed_mnar: Imputed data for MNAR case
        
        Returns:
        Dictionary with error metrics for both methods
        """
        if self.ground_truth_mcar is None or self.ground_truth_mnar is None:
            raise ValueError("Must call generate_missing() first")
            
        def calculate_metrics(pred: np.ndarray, true: np.ndarray, mask: np.ndarray) -> Dict[str, float]:
            pred_missing = pred[mask]
            mse = np.mean((pred_missing - true) ** 2)
            mae = np.mean(np.abs(pred_missing - true))
            rmse = np.sqrt(mse)
            return {
                #'MSE': mse,
                'MAE': mae,
                'RMSE': rmse
            }
        
        return {
            'MCAR': calculate_metrics(X_imputed_mcar, self.ground_truth_mcar, self.missing_mask_mcar),
            'MNAR': calculate_metrics(X_imputed_mnar, self.ground_truth_mnar, self.missing_mask_mnar)
        }

## Load data

In [9]:
SAMPLE = True
RESAMPLE = False
RESAMPLING = '10min'
TARGET = "p (mbar)"

if SAMPLE:
    n_rows = 100000
else:
    n_rows = None

MCAR_PROB=0.2
MNAR_SCALE=0.2

In [10]:
def resample_climate(original_df, every='1h'):

    resampled_df = (original_df
                     .group_by_dynamic("Date Time", every=every)
                     .agg(pl.all()
                          .mean()))

    return resampled_df

In [11]:
data = pl.read_parquet("/kaggle/input/climate-ts-imputation/climate.parquet", 
                        n_rows=n_rows)

if RESAMPLE:
    data = resample_climate(data, every=RESAMPLING)
    
# Assuming your data has a 'Date Time' column.
print("\nFull dataset date range:")
print(f"  Start: {data['Date Time'].min()}")
print(f"  End:   {data['Date Time'].max()}")

print("Data size for imputation and train: ", data.shape)


Full dataset date range:
  Start: 2009-01-01 00:10:00
  End:   2010-11-25 11:00:00
Data size for imputation and train:  (100000, 15)


## Testing PyGrinder in 1 year subset and visualize

In [12]:
if not SAMPLE:
    y1_data = data.filter(pl.col("Date Time").dt.year() == 2014)
else:
    y1_data = data.clone()

# Convert to numpy
y_y1 = y1_data.drop("Date Time").select(TARGET).to_numpy() # Save T to forecast
X_y1 = y1_data.drop(["Date Time", TARGET]).to_numpy()

# Scaling --> Difference between "mean" and "others" increase a lot.
from sklearn.preprocessing import StandardScaler
scalerX_y1 = StandardScaler().fit(X_y1)
scalery_y1 = StandardScaler().fit(y_y1)

X_y1 = scalerX_y1.transform(X_y1)
y_y1 = scalery_y1.transform(y_y1)

# Initialize with your 2D data
executor = PyGrinderExec(X_y1)

# Generate missing values (returns both MCAR and MNAR-t corrupted data and ground truth)
results = executor.generate_missing(
    mcar_prob=MCAR_PROB,           # 10% missing for MCAR
    mnar_cycle=20,           # MNAR-t parameters
    mnar_pos=10,
    mnar_scale=MNAR_SCALE
)

mcar_rate = calc_missing_rate(results['X_mcar'])
mnar_rate = calc_missing_rate(results['X_mnar'])
print(f"MCAR missing rate: {100*mcar_rate:.2f}%")
print(f"MNAR missing rate: {100*mnar_rate:.2f}%")

MCAR missing rate: 19.95%
MNAR missing rate: 20.18%


In [ ]:
def plot_missing_pattern(X: np.ndarray, 
                        title: str = "Missing Values Pattern",
                        figsize: tuple = (8, 5)):
    """
    Visualize missing values pattern in the dataset
    
    Parameters:
    X: Input array with missing values (NaN)
    title: Plot title
    figsize: Figure size (width, height)
    """
    # Create missing values mask (True where value is missing)
    missing_mask = np.isnan(X)
    
    # Calculate missing percentage
    missing_pct = np.mean(missing_mask) * 100
    
    # Create figure
    plt.figure(figsize=figsize)
    
    # Create heatmap
    sns.heatmap(missing_mask, 
                cbar=False,
                cmap='binary',
                yticklabels=False)
    
    # Customize plot
    plt.title(f"{title}\nTotal Missing: {missing_pct:.1f}%")
    plt.xlabel("Features")
    plt.ylabel("Date Time")
    
    # Show plot
    plt.tight_layout()
    plt.savefig(f"missing_pattern_{RESAMPLING}.png")
    plt.savefig(f"missing_pattern_{RESAMPLING}.svg")
    plt.show()

# For MCAR corrupted data
plot_missing_pattern(
    results['X_mcar'],
    title="MCAR Missing Pattern",
)

# For MNAR corrupted data
plot_missing_pattern(
    results['X_mnar'],
    title="MNAR Missing Pattern",
)

## Grinder whole data

In [ ]:
# Convert to numpy
y = data.drop("Date Time").select(TARGET).to_numpy() # Save T to forecast
X = data.drop(["Date Time", TARGET]).to_numpy()

# Scaling --> Difference between "mean" and "others" increase a lot.
from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler().fit(X)
scalery = StandardScaler().fit(y)

X = scalerX.transform(X)
y = scalery.transform(y)

# Initialize with 2D data
executor = PyGrinderExec(X)

# Generate missing values (returns both MCAR and MNAR-t corrupted data and ground truth)
results = executor.generate_missing(
    mcar_prob=MCAR_PROB,           # 20% missing for MCAR
    mnar_scale=MNAR_SCALE
)

# Access the corrupted datasets and ground truth
X_mcar = results['X_mcar']
X_mnar = results['X_mnar']
ground_truth_mcar = results['ground_truth_mcar']
ground_truth_mnar = results['ground_truth_mnar']

# Imputation

## Perform "simple" imputations (mean, interpolation, KNN)

In [ ]:
def linear_interpolate(X: np.ndarray) -> np.ndarray:
    """
    Performs linear interpolation on each column of X to fill missing values.
    
    Parameters
    ----------
    X : np.ndarray of shape (n_samples, n_features)
        Input data with missing values (NaN) you'd like to fill.

    Returns
    -------
    np.ndarray
        A copy of X with all missing values filled by linear interpolation.
    """
    # Convert to DataFrame for easy interpolation
    df = pd.DataFrame(X)
    
    # Apply linear interpolation along rows (axis=0)
    # limit_direction='both' handles NaNs at the start/end
    df_interpolated = df.interpolate(
        method="linear", 
        limit_direction="both", 
        axis=0
    )
    
    # If any NaNs remain (e.g., entire column was NaN), fill forward/backward
    df_interpolated = df_interpolated.ffill().bfill()
    
    # Return as numpy array
    return df_interpolated.values

In [ ]:
def perform_imputations(X_mcar: np.ndarray, X_mnar: np.ndarray, n_neighbors: int = 5):
    """
    Perform three imputation strategies: Simple (Mean), Interpolation, and KNN.
    
    Parameters
    ----------
    X_mcar : np.ndarray
        MCAR dataset with missing values
    X_mnar : np.ndarray
        MNAR dataset with missing values
    n_neighbors : int, default=5
        Number of neighbors for KNNImputer
    
    Returns
    -------
    dict
        Dictionary of imputed results for MCAR and MNAR with keys:
            'mean_mcar', 'mean_mnar'
            'interp_mcar', 'interp_mnar'
            'knn_mcar', 'knn_mnar'
    """
    # Initialize mean & KNN imputers
    mean_imputer = SimpleImputer(strategy='mean')
    knn_imputer = KNNImputer(n_neighbors=n_neighbors)
    
    # 1. Mean Imputation
    X_imputed_mcar_mean = mean_imputer.fit_transform(X_mcar)
    X_imputed_mnar_mean = mean_imputer.fit_transform(X_mnar)
    
    # 2. Interpolation Imputation 
    X_imputed_mcar_inter = linear_interpolate(X_mcar)
    X_imputed_mnar_inter = linear_interpolate(X_mnar)
    
    # 3. KNN Imputation
    X_imputed_mcar_knn = knn_imputer.fit_transform(X_mcar)
    X_imputed_mnar_knn = knn_imputer.fit_transform(X_mnar)
    
    return {
        'mean_mcar': X_imputed_mcar_mean,
        'mean_mnar': X_imputed_mnar_mean,
        'interp_mcar': X_imputed_mcar_inter,
        'interp_mnar': X_imputed_mnar_inter,
        'knn_mcar': X_imputed_mcar_knn,
        'knn_mnar': X_imputed_mnar_knn
    }

In [ ]:
# Perform "simple" imputations
imputed_results = perform_imputations(
    results['X_mcar'], 
    results['X_mnar'],
    n_neighbors=5
)

In [ ]:
imputed_results.keys()

# SAITS Imputation

To use **SAITS** on my single time series:

- I reshaped my single time series into multiple windows of shape `(N, window_size, features)` so SAITS could see multiple “mini-sequences” instead of one long array.  
- By doing this, the transformer-based model can better learn temporal dependencies and cross-feature relationships.  
- For a fast implementation and quick demo, I skipped the usual best practices—like creating a validation set and separately imputing the test set—and instead demonstrated SAITS directly on my training data.  
- After SAITS completed the imputation window-by-window, I averaged any overlapping predictions to restore the final shape.

In [ ]:
from pypots.imputation import SAITS
from pypots.utils.metrics import calc_mae

## Reshape corrupted data for SAITS

In [ ]:
def create_windows(
    X_2d: np.ndarray, 
    window_size: int, 
    step: int = 1
) -> np.ndarray:
    """
    Converts a 2D time-series array (T, F) into overlapping windows (N, window_size, F).

    Parameters:
    - X_2d (np.ndarray): The input 2D array with shape (T, F), where:
        * T: Number of time steps
        * F: Number of features
    - window_size (int): The size of each window (number of time steps per window).
    - step (int): The stride or step size for moving the window. Default is 1.
    """
    T, F = X_2d.shape
    windows = []
    for start in range(0, T - window_size + 1, step):
        end = start + window_size
        windows.append(X_2d[start:end, :])
    return np.array(windows)  # (N, window_size, F)

In [ ]:
WINDOW = 28 #(una semana)

## Train and impute

In [ ]:
def impute_and_evaluate(X_corrupted, X_original, window_size=28, epochs=10):
    """Imputes missing values in a corrupted dataset using SAITS and evaluates MAE."""

    # Create sliding windows for the corrupted and original datasets
    X_corrupted_3d = create_windows(X_corrupted, window_size=window_size, step=1)
    X_original_3d = create_windows(X_original, window_size=window_size, step=1)

    # Prepare dataset for SAITS
    dataset = {"X": X_corrupted_3d}

    # Initialize SAITS model
    saits = SAITS(
        n_steps=window_size,
        n_features=X_corrupted_3d.shape[2],
        n_layers=2,
        d_model=256,
        d_ffn=128,
        n_heads=4,
        d_k=64,
        d_v=64,
        dropout=0.1,
        epochs=epochs,
        diagonal_attention_mask=True,
    )

    # Train SAITS using the corrupted dataset
    saits.fit(dataset)

    # Perform imputation
    imputed_X = saits.impute(dataset)

    # Create a mask for artificially missing values (where X_corrupted_3d is NaN but X_original_3d is not)
    missing_mask_3d = np.isnan(X_corrupted_3d) & ~np.isnan(X_original_3d)

    # Calculate MAE using the mask
    mae = calc_mae(
        imputed_X,
        np.nan_to_num(X_original_3d),  # Replace NaNs with 0 (or another default value)
        missing_mask_3d
    )

    print(f"MAE (MCAR, {window_size} steps): {mae:.4f}")

    return imputed_X

In [ ]:
saits_imputed_mcar = impute_and_evaluate(X_mcar, X, window_size=WINDOW, epochs=10)

In [ ]:
saits_imputed_mnar = impute_and_evaluate(X_mnar, X, window_size=WINDOW, epochs=10)

## Reconstruct to original shape for comparison with "simple" methods

In [ ]:
def reconstruct_time_series_from_windows(
    windows_3d: np.ndarray, 
    T: int, 
    window_size: int, 
    step: int = 1
) -> np.ndarray:
    """
    Reconstructs a 2D array (T, F) from a 3D array (N, window_size, F), 
    assuming the same logic from `create_windows` was used with step=1 (or another step).
    A simple average is applied at each time position if it appears in multiple windows.
    """
    N, w_size, F = windows_3d.shape
    recon = np.zeros((T, F), dtype=np.float32)
    counts = np.zeros((T, 1), dtype=np.float32)
    
    idx = 0
    for start in range(0, T - window_size + 1, step):
        end = start + window_size
        recon[start:end, :] += windows_3d[idx, :, :]  # Sum window values
        counts[start:end, 0] += 1  # Track the number of contributions
        idx += 1
    
    # Avoid division by zero; counts should be >0 in the valid range
    nonzero_mask = counts[:, 0] != 0
    recon[nonzero_mask, :] /= counts[nonzero_mask, 0, np.newaxis]
    
    return recon

In [ ]:
saits_imputed_mcar_recon = reconstruct_time_series_from_windows(saits_imputed_mcar,
                                                             X.shape[0], #Original timesteps count
                                                             window_size=WINDOW, step=1)
saits_imputed_mnar_recon = reconstruct_time_series_from_windows(saits_imputed_mnar,
                                                             X.shape[0], #Original timesteps count
                                                             window_size=WINDOW, step=1)

# Evaluate methods

In [ ]:
# Evaluate methods
metrics_mean = executor.evaluate_imputation(
    imputed_results['mean_mcar'],
    imputed_results['mean_mnar']
)

metrics_interp = executor.evaluate_imputation(
    imputed_results['interp_mcar'],
    imputed_results['interp_mnar']
)

metrics_knn = executor.evaluate_imputation(
    imputed_results['knn_mcar'],
    imputed_results['knn_mnar']
)

metrics_saits = executor.evaluate_imputation(
    saits_imputed_mcar_recon,
    saits_imputed_mnar_recon
)

In [ ]:
print("-" * 60)
print(f"Imputation Metrics for Resampling = {RESAMPLING}, Data Shape = {X.shape}")
print("-" * 60)
print(f"{'Method':<15}{'MCAR_MAE':<12}{'MCAR_RMSE':<12}{'MNAR_MAE':<12}{'MNAR_RMSE':<12}")
print("-" * 60)

metrics_dict = {
        "Mean": metrics_mean,
        "Interpolation": metrics_interp,
        "KNN": metrics_knn,
        "SAITS": metrics_saits,
    }

print("-" * 60)

for method, metrics in metrics_dict.items():
    mcar_mae = metrics["MCAR"]["MAE"]
    mcar_rmse = metrics["MCAR"]["RMSE"]
    mnar_mae = metrics["MNAR"]["MAE"]
    mnar_rmse = metrics["MNAR"]["RMSE"]
    print(f"{method:<15}{mcar_mae:<12.6f}{mcar_rmse:<12.6f}{mnar_mae:<12.6f}{mnar_rmse:<12.6f}")

print("-" * 60)

In [ ]:
def plot_imputation_comparison(
    metrics_mean: dict, 
    metrics_interp: dict, 
    metrics_knn: dict,
    metrics_saits: dict,
    figsize: tuple = (12, 7)
):
    """
    Visualize imputation metrics comparison between Mean, Interpolation, KNN, and SAITS for both MCAR and MNAR.
    Uses a log scale on the y-axis.

    Parameters
    ----------
    metrics_mean : dict
        A dictionary with keys 'MCAR' and 'MNAR' for the Mean imputation metrics.
    metrics_interp : dict
        A dictionary with keys 'MCAR' and 'MNAR' for the Interpolation imputation metrics.
    metrics_knn : dict
        A dictionary with keys 'MCAR' and 'MNAR' for the KNN imputation metrics.
    metrics_saits : dict
        A dictionary with keys 'MCAR' and 'MNAR' for the SAITS imputation metrics.
    figsize : tuple
        Figure size (width, height) in inches.
    """
    plt.figure(figsize=figsize)
    
    # Extract metric names from the MCAR dictionary of metrics_mean
    metric_names = list(metrics_mean['MCAR'].keys())
    
    # Prepare values for each category
    values = {
        'MCAR (Mean)': list(metrics_mean['MCAR'].values()),
        'MCAR (Interp)': list(metrics_interp['MCAR'].values()),
        'MCAR (KNN)': list(metrics_knn['MCAR'].values()),
        'MCAR (SAITS)': list(metrics_saits['MCAR'].values()),
        'MNAR (Mean)': list(metrics_mean['MNAR'].values()),
        'MNAR (Interp)': list(metrics_interp['MNAR'].values()),
        'MNAR (KNN)': list(metrics_knn['MNAR'].values()),
        'MNAR (SAITS)': list(metrics_saits['MNAR'].values())
    }
    
    # Define positions for bars
    x = np.arange(len(metric_names))
    width = 0.1  # Adjust width to fit all bars properly

    # Use a colormap for better distinction
    cmap = plt.get_cmap("viridis")
    colors = [cmap(x_i) for x_i in np.linspace(0.2, 1.0, 8)]

    # Plot bars for each imputation method
    plt.bar(x - 3.5*width, values['MCAR (Mean)'], width, label='MCAR (Mean)', color=colors[0])
    plt.bar(x - 2.5*width, values['MCAR (Interp)'], width, label='MCAR (Interp)', color=colors[1])
    plt.bar(x - 1.5*width, values['MCAR (KNN)'], width, label='MCAR (KNN)', color=colors[2])
    plt.bar(x - 0.5*width, values['MCAR (SAITS)'], width, label='MCAR (SAITS)', color=colors[3])
    plt.bar(x + 0.5*width, values['MNAR (Mean)'], width, label='MNAR (Mean)', color=colors[4])
    plt.bar(x + 1.5*width, values['MNAR (Interp)'], width, label='MNAR (Interp)', color=colors[5])
    plt.bar(x + 2.5*width, values['MNAR (KNN)'], width, label='MNAR (KNN)', color=colors[6])
    plt.bar(x + 3.5*width, values['MNAR (SAITS)'], width, label='MNAR (SAITS)', color=colors[7])

    # Customize plot
    plt.title('Imputation Methods Comparison', pad=20)
    plt.xlabel('Metrics')
    plt.xticks(x, metric_names)
    plt.yscale("log")
    plt.legend()
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Create visualization
plot_imputation_comparison(metrics_mean, metrics_interp, metrics_knn, metrics_saits)

# Save results to load on Forecast notebook

In [ ]:
imputed_results['saits_mcar'] = saits_imputed_mcar_recon
imputed_results['saits_mnar'] = saits_imputed_mnar_recon

In [ ]:
def save_imputed_results(imputed_dict: dict, filename: str = f"imputed_results_{RESAMPLING}.npz"):
    np.savez_compressed(filename, **imputed_dict)
    print(f"Saved imputed results to {filename}")

save_imputed_results(imputed_results)

In [ ]:
def save_originals(original_data: pl.DataFrame, target: str, save_path: str = "./originals"):
    """
    Saves the train dataset as parquet files and the extracted numpy arrays as .npy files.

    Parameters:
    - data (pl.DataFrame): original dataset in Polars format.
    - target (str): Target column name.
    - save_path (str): Path to save the files.
    """
    import os
    os.makedirs(save_path, exist_ok=True)
    
    # Convert to numpy
    y = data.drop("Date Time").select(target).to_numpy()
    X = data.drop(["Date Time", target]).to_numpy()

    # Save Polars DataFrames as Parquet
    data.write_parquet(f"{save_path}/train_data.parquet")

    # Save numpy arrays
    np.save(f"{save_path}/X.npy", X)
    np.save(f"{save_path}/y.npy", y)

In [ ]:
save_originals(data, target=TARGET)